<a href="https://colab.research.google.com/github/rupesh0402/SentimentAnalysis/blob/main/sentiment_analysis_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

# Plotting Data to get insights

# Cleaning Data for Training

# Model training

# Checking Model Accuracy, Confusion matrix

# Plotting End results